<a href="https://colab.research.google.com/github/donmarcos/pharmacopia/blob/main/openfda/contradiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Logic for finding CONTRADICTIONS between 2 drugs**
###We find drug interactions for each drug apply NLP and compare it with active components of each other

In [1]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz (120.2 MB)


In [2]:
import spacy
import scispacy
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector
import json
import requests

In [3]:
nlp = spacy.load("en_core_sci_md")
#nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

In [4]:
#finding drug unique number
drugs = ['Amoxicillin','Prinivil']

In [5]:
def unique_number(drugs):
  unique_no = []
  for drug in drugs:
    url = "https://api.fda.gov/drug/drugsfda.json?search={}&limit=10".format(drug.upper())
    response = requests.get(url)
    for i in range(10):
      try:
        unii = response.json()['results'][i]['openfda']['unii'][0]
      except:
        pass
    unique_no.append(unii)
  return unique_no

In [6]:
uno = unique_number(drugs)
unique_number_one = uno[0]
unique_number_two = uno[1]

In [7]:
drug_fda = 'https://api.fda.gov/drug/drugsfda.json?search={}&limit=1'
drug_label = 'https://api.fda.gov/drug/label.json?search=drug_interactions:{}&limit=1'

In [8]:
query_one = drug_fda.format(unique_number_one)
response_one = requests.get(query_one)
query_two = drug_label.format(unique_number_two)
response_two = requests.get(query_two)

active_ingredients_one = ""
active_ingredients_two = ""


try:
  json_object_one = json.dumps(response_one.json()['results'][0]['products'][0]['active_ingredients'])
  for name in range(len(json.loads(json_object_one))):
    active_ingredients_one = json.loads(json_object_one)[name]['name']
except:
  pass
try:
  json_object_two = json.dumps(response_two.json()['results'][0]['products'][0]['active_ingredients'])
  for name in range(len(json.loads(json_object_two))):
      active_ingredients_two = json.loads(json_object_two)[name]['name']
except:
  pass

In [9]:
interactions_one = tuple()
interactions_two = tuple()
try:
  query_one = drug_label.format(drugs[0])
  response_one = requests.get(query_one)
  drug_interactions_one = json.dumps(response_one.json()['results'][0]['drug_interactions'][0])
  interactions_one = nlp(drug_interactions_one).ents
except:
  pass

try:
  query_two = drug_label.format(drugs[1])
  response_two = requests.get(query_two)
  drug_interactions_two = json.dumps(response_two.json()['results'][0]['drug_interactions'][0])
  interactions_two = nlp(drug_interactions_two).ents
except:
  pass


In [10]:
if active_ingredients_two=="" or len(interactions_one)==0:
  print("Information not found in database")
else:
    if len(list(set(interactions_one).intersection(list(active_ingredients_two))))==0:
      print("No hazardous reaction information was found between active components of drug two with the chemical interactions of drug one")

Information not found in database


In [11]:
if active_ingredients_one=="" or len(interactions_two)==0:
  print("Information not found in database")
else:
  if len(list(set(interactions_two).intersection(list(active_ingredients_one))))==0:
    print("No hazardous reaction information was found between active components of drug one with the chemical interactions of drug two")

No hazardous reaction information was found between active components of drug one with the chemical interactions of drug two
